In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

from const import DATA_DIR

# =========================
# LOAD DATA
# =========================
chornobyl_data_path = DATA_DIR / "chornobyl" / "data"
spatial_path = chornobyl_data_path / "1_Spatial_dataset.csv"
plutonium_path = chornobyl_data_path / "2_Plutonium_isotope_measurements.csv"

spatial = pd.read_csv(spatial_path)
plutonium = pd.read_csv(plutonium_path)

spatial.columns = spatial.columns.str.strip().str.lower()
plutonium.columns = plutonium.columns.str.strip().str.lower()

spatial = spatial.rename(columns={
    "code": "id",
    "latitude": "lat",
    "longitude": "lon"
})

PU_COL = "terrestrial_density_of_soil_contamination_with_239_240pu_kbq_m-2"

plutonium = plutonium.rename(columns={
    "code": "id",
    PU_COL: "pu_activity"
})

df = pd.merge(
    spatial[["id", "lat", "lon"]],
    plutonium[["id", "pu_activity"]],
    on="id",
    how="inner"
)

df["pu_activity"] = pd.to_numeric(df["pu_activity"], errors="coerce")
df = df.dropna()

# =========================
# DISTANCE FROM REACTOR
# =========================
# Coordinate del reattore (Chernobyl)
REACTOR_LAT = 51.389
REACTOR_LON = 30.099

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

df["distance_km"] = haversine(
    REACTOR_LAT, REACTOR_LON,
    df["lat"], df["lon"]
)

df["pu_log"] = np.log10(df["pu_activity"])

# =========================
# FIGURE
# =========================
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df["distance_km"],
        y=df["pu_log"],
        mode="markers",
        marker=dict(
            size=6,
            color=df["pu_log"],
            colorscale="Inferno",
            showscale=True,
            colorbar=dict(
                title="log₁₀ Pu-239/240<br>(kBq / m²)"
            ),
            opacity=0.75
        ),
        hovertemplate=(
            "Distance: %{x:.1f} km<br>"
            "log₁₀ Pu-239/240: %{y:.2f}"
            "<extra></extra>",
        ),
        showlegend=False
    )
)

# =========================
# LAYOUT
# =========================
fig.update_layout(
    title=dict(
        text="<b>No Safe Distance After Chernobyl</b><br>"
             "<span style='font-size:13px;color:#666'>Plutonium contamination vs distance from reactor (log scale)</span>",
        x=0.02
    ),
    xaxis_title="Distance from reactor (km)",
    yaxis_title="log₁₀ Pu-239/240 contamination (kBq / m²)",
    plot_bgcolor="white",
    paper_bgcolor="white",
    width=1000,
    height=550,
    margin=dict(l=70, r=70, t=90, b=70),
    font=dict(size=12),
    xaxis=dict(fixedrange=True),
    yaxis=dict(fixedrange=True)
)

# SOURCE
fig.add_annotation(
    text="<i>Source: UNSCEAR – Chernobyl Environmental Measurements (Pu-239/240)</i>",
    xref="paper", yref="paper",
    x=1, y=-0.18,
    showarrow=False,
    xanchor="right",
    font=dict(size=10, color="#888")
)

fig.show()


In [3]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "no-safe-distance-after-chernobyl.html")